# Vertex Nomination

## References

theory papers:
- theory of consistency and Bayes optimality: On consistent vertex nomination schemes
- dealing with corrupted networks (e.g., not one-to-one correspondence): Vertex Nomination, Consistent Estimation, and Adversarial Modification

applications papers:
- social networks: Vertex nomination via local neighborhood matching
- data associated with human trafficking: Vertex nomination schemes for membership prediction

this is the spectral vertex nomination stuff? 
https://microsoft.github.io/graspologic/tutorials/vertex_nomination/SpectralVertexNomination.html